In [1]:
import tensorflow as tf
import numpy as np

%load_ext tensorboard

In [20]:
class Dense(tf.Module):
  def __init__(self, in_features, out_features, name=None):
    super().__init__(name=name)
    self.w = tf.Variable(
      tf.random.normal([in_features, out_features]), name='w')
    self.w = tf.Variable(
      tf.Variable([[-3.14], [-2.31], [2.16]]), name='w')
    self.b = tf.Variable(tf.zeros([out_features]), name='b')
  def __call__(self, x):
    y = tf.matmul(x, self.w) + self.b
    return tf.nn.sigmoid(y)

dense = Dense(3, 1, 'test')
dense(tf.constant([[0.0288, -0.3256, 0.5925]]))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.8745173]], dtype=float32)>

In [14]:
dense.trainable_variables

(<tf.Variable 'b:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>,
 <tf.Variable 'w:0' shape=(3, 1) dtype=float32, numpy=
 array([[-0.72341204],
        [ 1.0436834 ],
        [-0.6476256 ]], dtype=float32)>)

In [2]:
class Dense(tf.Module):
  def __init__(self, in_features, out_features, name=None):
    super().__init__(name=name)
    self.w = tf.Variable(
      tf.random.normal([in_features, out_features]), name='w')
    #self.w = tf.Variable(
      #tf.Variable([[-3.14], [-2.31], [2.16]]), name='w')
    self.b = tf.Variable(tf.zeros([out_features]), name='b')
  def __call__(self, x):
    y = tf.matmul(x, self.w) + self.b
    return tf.nn.sigmoid(y)


class SequentialModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)

    self.dense_1 = Dense(in_features=3, out_features=3)
    self.dense_2 = Dense(in_features=3, out_features=1)

  @tf.function
  def __call__(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

my_model = SequentialModule(name="the_model")
my_model(tf.constant([[0.0288, -0.3256, 0.5925]]))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.3423126]], dtype=float32)>

In [29]:
chkp_path = "test/my_checkpoint"
checkpoint = tf.train.Checkpoint(model=dense)
checkpoint.write(chkp_path)

'test/my_checkpoint'

In [30]:
tf.train.list_variables(chkp_path)

[('_CHECKPOINTABLE_OBJECT_GRAPH', []),
 ('model/b/.ATTRIBUTES/VARIABLE_VALUE', [1]),
 ('model/w/.ATTRIBUTES/VARIABLE_VALUE', [3, 1])]

In [5]:
from datetime import datetime

# Set up logging.
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = "test/logs/func/%s" % stamp
writer = tf.summary.create_file_writer(logdir)

# Create a new model to get a fresh trace
# Otherwise the summary will not see the graph.
new_model = SequentialModule()

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
tf.summary.trace_on(graph=True, profiler=True)
# Call only one tf.function when tracing.
z = print(new_model(tf.constant([[0.0288, -0.3256, 0.5925]])))
with writer.as_default():
  tf.summary.trace_export(
      name="my_func_trace",
      step=0,
      profiler_outdir=logdir)

tf.Tensor([[0.7583965]], shape=(1, 1), dtype=float32)


In [8]:
%tensorboard --logdir test/logs/func

Reusing TensorBoard on port 6006 (pid 21156), started 0:01:56 ago. (Use '!kill 21156' to kill it.)

In [9]:
tf.saved_model.save(my_model, "test/the_saved_model")

INFO:tensorflow:Assets written to: test/the_saved_model\assets


In [11]:
new_model = tf.saved_model.load("test/the_saved_model")
new_model(tf.constant([[0.0288, -0.3256, 0.5925]]))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.3423126]], dtype=float32)>